In [253]:
"""%pip install mne --quiet
%pip install edfio --quiet
%pip install pymatreader --quiet
%pip install git+https://github.com/lina-usc/eog-learn.git --quiet --no-deps
%pip install mne-icalabel --quiet""";

Before running this analysis, the `run_eog_lstm_regression_mp.py` and `run_eog_lstm_ica_mp.py` scripts need to have been run so that the raw files are created on the disk locally.

In [1]:
import mne
from mne.io import BaseRaw
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

from tqdm.notebook import tqdm
import seaborn as sns
import eoglearn  # This is my package for this project

from pathlib import Path

2024-05-16 23:24:56.304203: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
def get_epochs(raw, events, baseline=(None, 0)):
  # this applies a baseline to the eyegaze channels which we dont generally want but okay for now i think
  return mne.Epochs(raw, events, tmin=-.2, tmax=1, preload=True, baseline=baseline, verbose=False)

def get_evoked(inst, picks, events=None, event_ids=("2", "5", "15"), baseline=(None, 0)):
  # see https://eoglearn.readthedocs.io/en/latest/auto_examples/plot_eegeyenet.html#sphx-glr-auto-examples-plot-eegeyenet-py
  # for stimuli locations
  if isinstance(inst, BaseRaw):
    assert events is not None, "Must pass in an events array to get evoked from Raw."
    inst = get_epochs(inst, events, baseline=baseline)
  if event_ids is None:
    return {kind: inst[kind].average().pick(picks) for kind in inst.event_id}
  return inst[event_ids].average().pick(picks)

#frontal = ["E19", "E11", "E4", "E12", "E5"]
#frontal = ["E19", "E11", "E4", "E12", "E5", "E24", "E124"]
#frontal = ["E8", "E9", "E15", "E22", "E25", "E1", "E2", "E3", "E10", "E16", "E18", "E23", "E26"]

In [4]:
def get_insts(fname_clean, event_ids=None):
    fname = Path(str(fname_clean).replace("clean", "ica"))
    if not fname.exists():
        return None

    evoked = {}
    raw = {}
    
    for kind in ["original", "clean", "noise", "ica"]:
        fname = Path(str(fname_clean).replace("clean", kind))
        raw[kind] = mne.io.read_raw_edf(fname, verbose=False)

        # Going through annotations because it is more robust than 
        # events (i.e., events are not transferable between raw objects
        # if these objects are not resampled at the same frequency). Resampling
        # of these different raw objects is done by different pieces of code.
        if kind == "original":
            annotations  = raw["original"].copy().drop_channels(['L-GAZE-X', 'L-GAZE-Y', 'L-AREA']).annotations

        raw[kind].set_annotations(annotations)
        events, _ = mne.events_from_annotations(raw[kind], regexp="^[0-9]*$", verbose=False)
        if kind == "original":        
            mapping = {"L-GAZE-X": ('eyegaze', 'px', 'left', 'x'),
                    "L-GAZE-Y": ("eyegaze", "px", "left", "y"),
                    "L-AREA": ("pupil", "au", "left")}
            mne.preprocessing.eyetracking.set_channel_types_eyetrack(raw["original"], mapping)
            evoked_eye = get_evoked(raw["original"], "eyegaze", events, event_ids)

        if "TIME" in raw[kind].ch_names:
            raw[kind].drop_channels(["TIME"])

        ch_names = [ch_name for ch_name in raw[kind].ch_names if ch_name[0] == "E"]
        raw[kind].set_channel_types(dict(zip(ch_names, 
                                    ["eeg"]*len(ch_names))))

        evoked[kind] = get_evoked(raw[kind], "eeg", events, event_ids)    

    return raw, evoked, evoked_eye

In [5]:
def rms(x):
  return np.sqrt(np.mean(x**2, axis=1))

def get_snrs(evoked_eye, evoked, event_id):
    ev_eye = evoked_eye[event_id].get_data()
    r = np.sqrt(ev_eye[0]**2 + ev_eye[1]**2)
    times = evoked["original"][event_id].times
    rt = times[(r > 0.05*r.max()) & (times > 0.1)][0]

    ev = {kind: evoked[kind][event_id].get_data() for kind in evoked}
    rt_masks = {"pre": evoked["original"][event_id].times < rt,
                "post": evoked["original"][event_id].times > rt}
    
    dfs = []
    dfs_topo = {("pre", "ica"): [], 
                ("pre", "clean"): [],
                ("post", "ica"): [], 
                ("post", "clean"): []}
    eeg_names = evoked["clean"][event_id].ch_names
    for condition, mask in rt_masks.items():
        signal = rms(ev["original"][:, mask])
        
        for kind in ["clean", "ica"]:
            noise = rms((ev["original"] - ev[kind])[:, mask])
            df = pd.DataFrame([10*np.log10(signal/noise)], columns=eeg_names)
            df["approach"] = kind
            df["condition"] = condition
            df["event_id"] = event_id
            dfs.append(df)

            nsr = rms(ev[kind][:, mask]) / rms(ev["original"][:, mask])
            percent_noise = 1 - nsr
            percent_noise *= 100

            data_dict = dict(list(zip(eeg_names, percent_noise)))
            df = pd.DataFrame([data_dict])
            df["event_id"] = event_id
            dfs_topo[(condition, kind)].append(df)

    return pd.concat(dfs), rt, {key: pd.concat(dfs_topo[key]) for key in dfs_topo}


In [7]:
montage = mne.channels.make_standard_montage('GSN-HydroCel-129')
snr_dfs = []
eeg_signals_dfs = []
et_signals_dfs = []
topo_dfs = {"clean": [], "ica": []}
topo_ev_dfs = {("pre", "ica"): [], 
                ("pre", "clean"): [],
                ("post", "ica"): [], 
                ("post", "clean"): []}

for fname_clean in tqdm(list(Path("processed").glob("*_clean.edf"))):
    insts = get_insts(fname_clean)
    if insts is None:
        continue
    raw, evoked, evoked_eye = insts


    # TOPOMAP analysis
    eeg_names = raw["original"].copy().pick("eeg").ch_names

    nsample = min(len(raw["original"].times), len(raw["ica"])) 
    # Could probably removed once data are recomputed.
    signal = np.sqrt(np.mean(raw["original"].get_data(picks=eeg_names)[:, :nsample]**2, axis=1))

    for kind in topo_dfs:
        noise = raw["original"].get_data(picks=eeg_names)[:, :nsample] - raw[kind].get_data(picks=eeg_names)[:, :nsample]
        noise = np.sqrt(np.mean(noise**2, axis=1)) 
    
        nsr = noise / signal
        percent_noise = nsr
        percent_noise *= 100

        data_dict = dict(list(zip(eeg_names, percent_noise)))
        df = pd.DataFrame([data_dict])
        df["subject"], df["run"] = fname_clean.name.split("_")[:2]
        topo_dfs[kind].append(df)


    for event_id in evoked["original"]:
        # SNR analysis
        df, rt, topo_ev = get_snrs(evoked_eye, evoked, event_id)
        df["subject"], df["run"] = fname_clean.name.split("_")[:2]
        df["rt"] = rt
        snr_dfs.append(df)


        for key in topo_ev_dfs:
            topo_ev[key]["subject"], topo_ev[key]["run"] = fname_clean.name.split("_")[:2]
            topo_ev_dfs[key].append(topo_ev[key])

        # ERP analysis
        # eeg
        for key, kind in zip(['original', 'clean', 'ica'], ['original', 'clean-et', 'clean-ica']):
            sigs = evoked[key][event_id].get_data()
            ch_names = evoked[key][event_id].ch_names
            for ch_name, sig in zip(ch_names, sigs):
                df = pd.DataFrame({
                    'times': evoked[key][event_id].times,
                    "amp": sig
                })
                df["subject"], df["run"] = fname_clean.name.split("_")[:2]
                df["kind"] = kind
                df["event_id"] = event_id
                df["ch_name"] = ch_name
                eeg_signals_dfs.append(df)

        # et
        ev_eye = evoked_eye[event_id].get_data()
        df = pd.DataFrame({
            'times': evoked["original"][event_id].times,
            'eye-x': ev_eye[0],
            'eye-y': ev_eye[1]
        })
        df = df.melt(id_vars="times", value_name="amp", var_name="ch_name")
        df["subject"], df["run"] = fname_clean.name.split("_")[:2]
        df["event_id"] = event_id
        et_signals_dfs.append(df)


eeg_signals_df = pd.concat(eeg_signals_dfs)
et_signals_df = pd.concat(et_signals_dfs)
snr_df = pd.concat(snr_dfs)
topo_df = {kind: pd.concat(topo_dfs[kind]) for kind in topo_dfs}
topo_ev_df = {key: pd.concat(topo_ev_dfs[key]) for key in topo_ev_dfs}

  0%|          | 0/176 [00:00<?, ?it/s]

/var/folders/3k/220tdhsn33709gq9ylr8c6kh0000gp/T/ipykernel_70937/340364400.py:20: RuntimeWarning: Omitted 1 annotation(s) that were outside data range.
  raw[kind].set_annotations(annotations)
/var/folders/3k/220tdhsn33709gq9ylr8c6kh0000gp/T/ipykernel_70937/340364400.py:20: RuntimeWarning: Omitted 1 annotation(s) that were outside data range.
  raw[kind].set_annotations(annotations)
/var/folders/3k/220tdhsn33709gq9ylr8c6kh0000gp/T/ipykernel_70937/340364400.py:20: RuntimeWarning: Omitted 1 annotation(s) that were outside data range.
  raw[kind].set_annotations(annotations)
/var/folders/3k/220tdhsn33709gq9ylr8c6kh0000gp/T/ipykernel_70937/340364400.py:20: RuntimeWarning: Omitted 1 annotation(s) that were outside data range.
  raw[kind].set_annotations(annotations)
/var/folders/3k/220tdhsn33709gq9ylr8c6kh0000gp/T/ipykernel_70937/340364400.py:20: RuntimeWarning: Omitted 1 annotation(s) that were outside data range.
  raw[kind].set_annotations(annotations)
/var/folders/3k/220tdhsn33709gq9ylr

In [8]:
# Saving as xarrays

eeg_signals_df.set_index(["times", "kind", "event_id", "ch_name", "subject", "run"]).to_xarray().to_netcdf("eeg_signals.netcdf")

et_signals_df.set_index(["times", "ch_name", "subject", "run", "event_id"]).to_xarray().to_netcdf("et_signals.netcdf")

snr_df.melt(id_vars=["approach", "condition", "event_id", "subject", "run", "rt"], var_name="ch_name", value_name="snr")\
      .set_index(["approach", "condition", "event_id", "subject", "run", "ch_name"]).to_xarray().to_netcdf("snr.netcdf")

topo_df["clean"]["kind"] = "clean"
topo_df["ica"]["kind"] = "ica"
pd.concat(topo_df.values()).melt(id_vars=["subject", "run", "kind"], var_name="ch_name", value_name="percent")\
      .set_index(["subject", "run", "ch_name", "kind"]).to_xarray().to_netcdf("topo_raw.netcdf")

for condition, kind in  topo_ev_df:
    topo_ev_df[(condition, kind)]["condition"] = condition
    topo_ev_df[(condition, kind)]["kind"] = kind

pd.concat(topo_ev_df.values()).melt(id_vars=["subject", "run", "kind", "event_id", "condition"], var_name="ch_name", value_name="percent")\
      .set_index(["subject", "run", "ch_name", "kind", "event_id", "condition"]).to_xarray().to_netcdf("topo_erp.netcdf")